In [1]:
import pandas as pd
import numpy as np

In [2]:
#User features 
user_features = pd.read_csv("csvfiles/user_features_train.csv")
user_features_test = pd.read_csv("csvfiles/user_features_test.csv")


In [3]:
#Users meta data 
users = pd.read_csv("csvfiles/users_train.csv")
users_test = pd.read_csv("csvfiles/users_test.csv")

In [4]:
#Target_Train 
target = pd.read_csv("csvfiles/targets_train.csv")

In [5]:
user_features.head()

,ID,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,...,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,0,3.314099,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,1.681524,True,False,False,False,False,False,True,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10.718750,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,5.100000,True,True,True,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,2.091409,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
user_features_test.head()

,ID,first_prediction,RetentionD0,RetentionD1,RetentionD2,RetentionD3,RetentionD4,RetentionD5,RetentionD6,RetentionD7,...,IAPRevenueD6,IAPRevenueD7,IAPRevenueD8,IAPRevenueD9,IAPRevenueD10,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15
0,878594,1.444805,True,True,True,True,True,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,878595,9.147972,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,878596,40.731158,True,False,False,False,False,False,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,878597,4.967959,True,False,False,True,False,True,True,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,878598,2.445842,True,False,True,True,True,True,False,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
users.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,unityads_int
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,applovin_int
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,applovin_int
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,NaN
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,applovin_int


In [8]:
users_test.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,ad_network
0,878594,2024-05-12,1715478163668000,1715467363668000,Argentina,Android,mobile,Motorola,Moto G32,False,applovin_int
1,878595,2024-01-26,1706254855890000,1706233255890000,Mexico,Android,mobile,OnePlus,Nord N20 SE,False,applovin_int
2,878596,2024-02-13,1707778260263000,1707781860263000,France,Android,mobile,Motorola,moto g13,False,applovin_int
3,878597,2024-02-21,1708530744695000,1708519944695000,Brazil,Android,mobile,Samsung,Galaxy A03,False,applovin_int
4,878598,2024-05-03,1714706093405000,1714688093405000,Peru,Android,mobile,Xiaomi,Redmi 13C,False,applovin_int


In [9]:
target.head()

,ID,TARGET
0,0,0.000000
1,1,0.018892
2,2,0.000000
3,3,0.046650
4,4,0.014680


In [10]:
user_features.isna().sum()

ID                      0
first_prediction    25735
RetentionD0             0
RetentionD1             0
RetentionD2             0
                    ...  
IAPRevenueD11           0
IAPRevenueD12           0
IAPRevenueD13           0
IAPRevenueD14           0
IAPRevenueD15           0
Length: 76, dtype: int64

In [11]:
user_features_test.isna().sum()

ID                      0
first_prediction    17278
RetentionD0             0
RetentionD1             0
RetentionD2             0
                    ...  
IAPRevenueD11           0
IAPRevenueD12           0
IAPRevenueD13           0
IAPRevenueD14           0
IAPRevenueD15           0
Length: 76, dtype: int64

In [12]:
target.isna().sum()

ID        0
TARGET    0
dtype: int64

In [13]:
users.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           82
platform                           0
device_category                    0
device_brand                    5840
device_model                       0
has_ios_att_permission             0
ad_network                    310470
dtype: int64

In [14]:
users_test.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           56
platform                           0
device_category                    0
device_brand                    3927
device_model                       0
has_ios_att_permission             0
ad_network                    206591
dtype: int64

In [15]:
users["has_ios_att_permission"].value_counts()

has_ios_att_permission
False    764921
True     113673
Name: count, dtype: int64

In [16]:
users["device_brand"].value_counts()

device_brand
Apple          363623
Samsung        225831
Xiaomi          82158
Motorola        76009
OPPO            22920
                ...  
Aiprotablet         1
Fossibot            1
Meitu               1
Tagital             1
Movisun             1
Name: count, Length: 295, dtype: int64

In [17]:
users["ID"]

0              0
1              1
2              2
3              3
4              4
           ...  
878589    878589
878590    878590
878591    878591
878592    878592
878593    878593
Name: ID, Length: 878594, dtype: int64

In [18]:
merged_df = pd.merge(users, user_features, on = "ID")
df = pd.merge(merged_df, target, on="ID")

In [19]:
test_df = pd.merge(users_test, user_features_test, on = "ID")

In [20]:
#Leaving outside the players that uninstalled the game

retention_columns = [f"RetentionD{i}" for i in range(0, 16)]

#A function to calculate last active day of player

def last_active_day(row):
    for day in range(15,-1,-1):
        if row[f"RetentionD{day}"] == True:
            return day
    return 0 


In [21]:
df["Last Active Day"] = df.apply(last_active_day, axis=1)

In [22]:
test_df["Last Active Day"] = test_df.apply(last_active_day, axis=1)

In [23]:
df["Last Active Day"].head(20)

0     12
1      9
2      0
3      2
4      0
5      2
6      2
7      0
8      6
9      1
10     0
11     1
12    15
13     7
14     2
15     8
16     0
17     0
18     0
19     9
Name: Last Active Day, dtype: int64

In [24]:
test_df["Last Active Day"].head(20)

0      4
1      0
2      0
3      9
4      5
5      0
6      9
7      0
8     13
9     14
10     1
11     1
12     2
13     1
14    13
15    15
16     0
17     1
18     5
19     0
Name: Last Active Day, dtype: int64

In [25]:
def count_retention_values(row):
    true_count = sum(row[col] == True for col in retention_columns)
    false_count = sum(row[col] == False for col in retention_columns)
    return pd.Series([true_count, false_count], index=['True_count', 'False_count'])

In [26]:
#8. günden sonra oynaması lazım 5 günden az oynamaması lazım

df[['True_count', 'False_count']] = df.apply(count_retention_values, axis=1)

In [27]:
test_df[['True_count', 'False_count']] = test_df.apply(count_retention_values, axis=1)

In [28]:
def active_after_day8(row):
    return any(row[f"RetentionD{i}"] == True for i in range(8, 16))

In [29]:
def active_in_first_8_days(row):
    return sum(row[f"RetentionD{i}"] == True for i in range(0, 8)) >= 5

In [30]:
df['Active after 8'] = df.apply(active_after_day8, axis=1)
df['Active in first 8 days'] = df.apply(active_in_first_8_days, axis=1)

In [31]:
test_df['Active after 8'] = test_df.apply(active_after_day8, axis=1)
test_df['Active in first 8 days'] = test_df.apply(active_in_first_8_days, axis=1)

In [32]:
filtered_df = df[(df['Active after 8'] == True) | ((df["Active after 8"] == False) & (df['Active in first 8 days'] == True))]

In [33]:
filtered_test_df = test_df[(test_df['Active after 8'] == True) | ((test_df["Active after 8"] == False) & (test_df['Active in first 8 days'] == True))]

In [34]:
excluded_df = df[~df['ID'].isin(filtered_df['ID'])]

In [35]:
excluded_test_df = test_df[~test_df['ID'].isin(filtered_test_df['ID'])]

In [36]:
df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,...,0.0,0.0,0.0,0.0,0.000000,12,2,14,True,False
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,...,0.0,0.0,0.0,0.0,0.018892,9,4,12,True,False
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,...,0.0,0.0,0.0,0.0,0.000000,0,1,15,False,False
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,...,0.0,0.0,0.0,0.0,0.046650,2,3,13,False,False
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,...,0.0,0.0,0.0,0.0,0.014680,0,1,15,False,False


In [37]:
excluded_df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
2,2,2024-03-18,1710731043082000,1710720243082000,Brazil,Android,mobile,Xiaomi,Redmi 12,False,...,0.0,0.0,0.0,0.0,0.00000,0,1,15,False,False
3,3,2024-03-03,1709455862260000,1709441462260000,Dominican Republic,iOS,mobile,Apple,iPhone 11 Pro Max,False,...,0.0,0.0,0.0,0.0,0.04665,2,3,13,False,False
4,4,2024-04-30,1714482477190000,1714464477190000,Ecuador,Android,mobile,Motorola,Moto E22,False,...,0.0,0.0,0.0,0.0,0.01468,0,1,15,False,False
5,5,2024-04-23,1713905887691000,1713887887691000,Colombia,Android,mobile,Xiaomi,Redmi Note 8,False,...,0.0,0.0,0.0,0.0,0.00000,2,3,13,False,False
6,6,2024-03-30,1711829156974000,1711814756974000,Venezuela,Android,mobile,Samsung,Galaxy S20 FE,False,...,0.0,0.0,0.0,0.0,0.00000,2,2,14,False,False


In [38]:
excluded_test_df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
1,878595,2024-01-26,1706254855890000,1706233255890000,Mexico,Android,mobile,OnePlus,Nord N20 SE,False,...,0.0,0.0,0.0,0.0,0.0,0,1,15,False,False
2,878596,2024-02-13,1707778260263000,1707781860263000,France,Android,mobile,Motorola,moto g13,False,...,0.0,0.0,0.0,0.0,0.0,0,1,15,False,False
5,878599,2024-03-18,1710765344672000,1710765344672000,United Kingdom,Android,mobile,Samsung,Galaxy A71,False,...,0.0,0.0,0.0,0.0,0.0,0,1,15,False,False
7,878601,2024-04-15,1713201299344000,1713208499344000,South Africa,Android,mobile,Samsung,Galaxy A13,False,...,0.0,0.0,0.0,0.0,0.0,0,1,15,False,False
10,878604,2024-02-02,1706897925481000,1706901525481000,France,Android,mobile,Huawei,P20 Lite,False,...,0.0,0.0,0.0,0.0,0.0,1,2,14,False,False


In [39]:
filtered_df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,TARGET,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
0,0,2024-03-02,1709355895042000,1709334295042000,Mexico,Android,mobile,Xiaomi,Redmi A2,False,...,0.0,0.0,0.0,0.0,0.000000,12,2,14,True,False
1,1,2024-03-19,1710824539731000,1710806539731000,Peru,Android,mobile,Samsung,Galaxy A13,False,...,0.0,0.0,0.0,0.0,0.018892,9,4,12,True,False
12,12,2024-05-04,1714783839460000,1714791039460000,Germany,iOS,mobile,Apple,iPhone 14 Pro Max,False,...,0.0,0.0,0.0,0.0,1.265949,15,7,9,True,False
15,15,2024-02-05,1707146971878000,1707166771878000,India,Android,mobile,Xiaomi,Redmi 12 5G,False,...,0.0,0.0,0.0,0.0,0.044076,8,2,14,True,False
19,19,2024-01-06,1704551875588000,1704541075588000,Brazil,Android,mobile,Motorola,Moto G84 5G,False,...,0.0,0.0,0.0,0.0,0.046317,9,4,12,True,False


In [40]:
filtered_test_df.head()

,ID,first_open_date,first_open_timestamp,local_first_open_timestamp,country,platform,device_category,device_brand,device_model,has_ios_att_permission,...,IAPRevenueD11,IAPRevenueD12,IAPRevenueD13,IAPRevenueD14,IAPRevenueD15,Last Active Day,True_count,False_count,Active after 8,Active in first 8 days
0,878594,2024-05-12,1715478163668000,1715467363668000,Argentina,Android,mobile,Motorola,Moto G32,False,...,0.0,0.0,0.0,0.0,0.0,4,5,11,False,True
3,878597,2024-02-21,1708530744695000,1708519944695000,Brazil,Android,mobile,Samsung,Galaxy A03,False,...,0.0,0.0,0.0,0.0,0.0,9,6,10,True,False
4,878598,2024-05-03,1714706093405000,1714688093405000,Peru,Android,mobile,Xiaomi,Redmi 13C,False,...,0.0,0.0,0.0,0.0,0.0,5,5,11,False,True
6,878600,2024-03-07,1709836239479000,1709818239479000,Canada,Android,mobile,Samsung,Galaxy A32 5G,False,...,0.0,0.0,0.0,0.0,0.0,9,4,12,True,False
8,878602,2024-04-01,1711989085362000,1711978285362000,Argentina,Android,mobile,Samsung,Galaxy A30s,False,...,0.0,0.0,0.0,0.0,0.0,13,8,8,True,True


In [41]:
def calculate_15_day_revenue(row):
    ad_revenue = sum(row[f"AdRevenueD{i}"] for i in range(0, 16) if row[f"AdRevenueD{i}"] is not None)
    iap_revenue = sum(row[f"IAPRevenueD{i}"] for i in range(0, 16) if row[f"IAPRevenueD{i}"] is not None)
    return ad_revenue + iap_revenue

excluded_df['15_day_revenue'] = excluded_df.apply(calculate_15_day_revenue, axis=1)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\2501875321.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_df['15_day_revenue'] = excluded_df.apply(calculate_15_day_revenue, axis=1)


In [42]:
excluded_test_df['15_day_revenue'] = excluded_test_df.apply(calculate_15_day_revenue, axis=1)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3494311871.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  excluded_test_df['15_day_revenue'] = excluded_test_df.apply(calculate_15_day_revenue, axis=1)


In [43]:
excluded_df["15_day_revenue"].head()

2    0.00000
3    0.04665
4    0.01468
5    0.00000
6    0.00000
Name: 15_day_revenue, dtype: float64

In [44]:
excluded_test_df["15_day_revenue"].head()

1     0.000000
2     0.000000
5     0.106975
7     0.000000
10    0.000000
Name: 15_day_revenue, dtype: float64

In [45]:
filtered_df.drop(columns=['Last Active Day',
       'True_count', 'False_count', 'Active after 8',
       'Active in first 8 days'], inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3409789487.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=['Last Active Day',


In [46]:
filtered_test_df.drop(columns=['Last Active Day',
       'True_count', 'False_count', 'Active after 8',
       'Active in first 8 days'], inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\739109439.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df.drop(columns=['Last Active Day',


In [47]:
filtered_df.columns

Index(['ID', 'first_open_date', 'first_open_timestamp',
       'local_first_open_timestamp', 'country', 'platform', 'device_category',
       'device_brand', 'device_model', 'has_ios_att_permission', 'ad_network',
       'first_prediction', 'RetentionD0', 'RetentionD1', 'RetentionD2',
       'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6',
       'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10',
       'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14',
       'RetentionD15', 'LevelAdvancedCountD0', 'LevelAdvancedCountD1',
       'LevelAdvancedCountD2', 'LevelAdvancedCountD3', 'LevelAdvancedCountD4',
       'LevelAdvancedCountD5', 'LevelAdvancedCountD6', 'LevelAdvancedCountD7',
       'LevelAdvancedCountD8', 'LevelAdvancedCountD9', 'LevelAdvancedCountD10',
       'LevelAdvancedCountD11', 'LevelAdvancedCountD12',
       'LevelAdvancedCountD13', 'LevelAdvancedCountD14',
       'LevelAdvancedCountD15', 'Level_1_Duration', 'Level_2_Duration',
       'Level

In [48]:
users.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           82
platform                           0
device_category                    0
device_brand                    5840
device_model                       0
has_ios_att_permission             0
ad_network                    310470
dtype: int64

In [49]:
user_features.isna().sum()

ID                      0
first_prediction    25735
RetentionD0             0
RetentionD1             0
RetentionD2             0
                    ...  
IAPRevenueD11           0
IAPRevenueD12           0
IAPRevenueD13           0
IAPRevenueD14           0
IAPRevenueD15           0
Length: 76, dtype: int64

In [50]:
filtered_df["first_prediction"].isna().sum()

4876

In [51]:
filtered_df.isna().sum()

ID                             0
first_open_date                0
first_open_timestamp           0
local_first_open_timestamp     0
country                       57
                              ..
IAPRevenueD12                  0
IAPRevenueD13                  0
IAPRevenueD14                  0
IAPRevenueD15                  0
TARGET                         0
Length: 87, dtype: int64

In [52]:
filtered_df["country"].isna().sum()

57

In [53]:
filtered_df["device_brand"].isna().sum()

1527

In [54]:
filtered_df["ad_network"].isna().sum()

117625

In [55]:
filtered_df["ad_network"].fillna("organic_user", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3164673121.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df["ad_network"].fillna("organic_user", inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3164673121.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["ad_network"].fillna("organic_user", inplace

In [56]:
df["country"].mode()

0    United States
Name: country, dtype: object

In [57]:
filtered_df["country"].fillna("United States", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\2705615632.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df["country"].fillna("United States", inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\2705615632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["country"].fillna("United States", inplace=Tru

In [58]:
df["device_brand"].mode()

0    Apple
Name: device_brand, dtype: object

In [59]:
filtered_df["device_brand"].fillna("Apple", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\782682860.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df["device_brand"].fillna("Apple", inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\782682860.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["device_brand"].fillna("Apple", inplace=True)


In [60]:
df["first_prediction"].mean()

37.612360277089614

In [61]:
filtered_df["first_prediction"].fillna(37.612360277089614, inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\2682018155.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df["first_prediction"].fillna(37.612360277089614, inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\2682018155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df["first_prediction"].fillna(37.6123

In [62]:
filtered_df["country"].value_counts()

country
Brazil                    41835
Mexico                    40400
United States             36608
Argentina                 23063
Colombia                  15688
                          ...  
Papua New Guinea              1
Sint Maarten                  1
Kiribati                      1
Turks & Caicos Islands        1
Lesotho                       1
Name: count, Length: 206, dtype: int64

In [63]:
filtered_df.isna().sum()

ID                            0
first_open_date               0
first_open_timestamp          0
local_first_open_timestamp    0
country                       0
                             ..
IAPRevenueD12                 0
IAPRevenueD13                 0
IAPRevenueD14                 0
IAPRevenueD15                 0
TARGET                        0
Length: 87, dtype: int64

In [64]:
for level in range(1,11):
    filtered_df[f'Level_{level}_Duration'].fillna(0, inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3632186599.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_df[f'Level_{level}_Duration'].fillna(0, inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3632186599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df[f'Level_{level}_Duration'].fillna(0, inplace

In [65]:
filtered_test_df.columns

Index(['ID', 'first_open_date', 'first_open_timestamp',
       'local_first_open_timestamp', 'country', 'platform', 'device_category',
       'device_brand', 'device_model', 'has_ios_att_permission', 'ad_network',
       'first_prediction', 'RetentionD0', 'RetentionD1', 'RetentionD2',
       'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6',
       'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10',
       'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14',
       'RetentionD15', 'LevelAdvancedCountD0', 'LevelAdvancedCountD1',
       'LevelAdvancedCountD2', 'LevelAdvancedCountD3', 'LevelAdvancedCountD4',
       'LevelAdvancedCountD5', 'LevelAdvancedCountD6', 'LevelAdvancedCountD7',
       'LevelAdvancedCountD8', 'LevelAdvancedCountD9', 'LevelAdvancedCountD10',
       'LevelAdvancedCountD11', 'LevelAdvancedCountD12',
       'LevelAdvancedCountD13', 'LevelAdvancedCountD14',
       'LevelAdvancedCountD15', 'Level_1_Duration', 'Level_2_Duration',
       'Level

In [66]:
filtered_test_df.isna().sum()

ID                             0
first_open_date                0
first_open_timestamp           0
local_first_open_timestamp     0
country                       37
                              ..
IAPRevenueD11                  0
IAPRevenueD12                  0
IAPRevenueD13                  0
IAPRevenueD14                  0
IAPRevenueD15                  0
Length: 86, dtype: int64

In [67]:
users_test.isna().sum()

ID                                 0
first_open_date                    0
first_open_timestamp               0
local_first_open_timestamp         0
country                           56
platform                           0
device_category                    0
device_brand                    3927
device_model                       0
has_ios_att_permission             0
ad_network                    206591
dtype: int64

In [68]:
filtered_test_df["country"].fillna("United States", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\352407144.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_test_df["country"].fillna("United States", inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\352407144.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df["country"].fillna("United States", inp

In [69]:
filtered_test_df["country"].value_counts()

country
Brazil                      27997
Mexico                      27337
United States               24096
Argentina                   15565
India                       10626
                            ...  
Central African Republic        1
Macao                           1
South Sudan                     1
Eswatini                        1
Cook Islands                    1
Name: count, Length: 205, dtype: int64

In [70]:
filtered_test_df["device_brand"].fillna("Apple", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\1012357556.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_test_df["device_brand"].fillna("Apple", inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\1012357556.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df["device_brand"].fillna("Apple", inplace

In [71]:
filtered_test_df["ad_network"].fillna("organic_user", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3502400106.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_test_df["ad_network"].fillna("organic_user", inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3502400106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df["ad_network"].fillna("organic_user

In [72]:
test_df["first_prediction"].mean()

37.54019300786987

In [73]:
filtered_test_df["first_prediction"].fillna(37.54019300786987, inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\1185975934.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_test_df["first_prediction"].fillna(37.54019300786987, inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\1185975934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df["first_prediction"].filln

In [74]:
filtered_test_df["ad_network"].fillna("organic_user", inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\3502400106.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df["ad_network"].fillna("organic_user", inplace=True)


In [75]:
for level in range(1,11):
    filtered_test_df[f'Level_{level}_Duration'].fillna(0, inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\4256963988.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  filtered_test_df[f'Level_{level}_Duration'].fillna(0, inplace=True)
C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\4256963988.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_test_df[f'Level_{level}_Duration'].fillna(

In [76]:
filtered_test_df.isna().sum()

ID                            0
first_open_date               0
first_open_timestamp          0
local_first_open_timestamp    0
country                       0
                             ..
IAPRevenueD11                 0
IAPRevenueD12                 0
IAPRevenueD13                 0
IAPRevenueD14                 0
IAPRevenueD15                 0
Length: 86, dtype: int64

In [77]:
filtered_df.columns

Index(['ID', 'first_open_date', 'first_open_timestamp',
       'local_first_open_timestamp', 'country', 'platform', 'device_category',
       'device_brand', 'device_model', 'has_ios_att_permission', 'ad_network',
       'first_prediction', 'RetentionD0', 'RetentionD1', 'RetentionD2',
       'RetentionD3', 'RetentionD4', 'RetentionD5', 'RetentionD6',
       'RetentionD7', 'RetentionD8', 'RetentionD9', 'RetentionD10',
       'RetentionD11', 'RetentionD12', 'RetentionD13', 'RetentionD14',
       'RetentionD15', 'LevelAdvancedCountD0', 'LevelAdvancedCountD1',
       'LevelAdvancedCountD2', 'LevelAdvancedCountD3', 'LevelAdvancedCountD4',
       'LevelAdvancedCountD5', 'LevelAdvancedCountD6', 'LevelAdvancedCountD7',
       'LevelAdvancedCountD8', 'LevelAdvancedCountD9', 'LevelAdvancedCountD10',
       'LevelAdvancedCountD11', 'LevelAdvancedCountD12',
       'LevelAdvancedCountD13', 'LevelAdvancedCountD14',
       'LevelAdvancedCountD15', 'Level_1_Duration', 'Level_2_Duration',
       'Level

In [78]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import mean_squared_error

In [79]:
X = filtered_df.drop(columns=['TARGET', "first_open_date"])
y = filtered_df['TARGET']

In [80]:
X_test = filtered_test_df.drop(columns=["first_open_date"])

In [81]:
categorical_features = ['country', 'platform', 'device_category', 'device_brand', 'device_model', 'ad_network']

In [82]:
label_encoders = {}
for column in categorical_features:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column].astype(str))
    label_encoders[column] = le

In [83]:
label_encoders = {}
for column in categorical_features:
    le = LabelEncoder()
    X_test[column] = le.fit_transform(X_test[column].astype(str))
    label_encoders[column] = le

In [84]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [85]:
train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features, free_raw_data=False)
val_data = lgb.Dataset(X_val, label=y_val, categorical_feature=categorical_features, reference=train_data, free_raw_data=False)

In [86]:
param_grid = {
    'learning_rate': [0.01, 0.035, 0.1],
    'num_leaves': [31, 50, 70],
    'feature_fraction': [0.8, 0.9, 1.0],
    'bagging_fraction': [0.8, 0.9, 1.0],
    'bagging_freq': [5, 10, 15]
}

# Modeli tanımla
lgbm = lgb.LGBMRegressor(objective='regression', metric='rmse')


grid = GridSearchCV(lgbm, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid.fit(X_train, y_train)

print(f"Best parameters found: {grid.best_params_}")

Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.044642 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGB

In [98]:
best_params = {'bagging_fraction': 0.8, 'bagging_freq': 10, 'feature_fraction': 0.9, 'learning_rate': 0.1, 'num_leaves': 70}
lgbm_best = lgb.LGBMRegressor(objective='regression', metric='rmse', **best_params)

In [99]:
lgbm_best.fit(X_train, y_train)

y_pred = lgbm_best.predict(X_val)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.037058 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 11010
[LightGBM] [Info] Number of data poin

In [100]:
test_rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print(f"Test RMSE: {test_rmse}")

Test RMSE: 2.698810192534511


In [101]:
y_test_pred = lgbm_best.predict(X_test)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


In [184]:
ids = filtered_test_df['ID'].values

In [185]:
predictions_df = pd.DataFrame({
    'ID': ids,
    'TARGET': y_test_pred
})

In [186]:
predictions_df_df = excluded_test_df[["ID", "15_day_revenue"]]

In [200]:
predictions_df_df.rename(columns={"15_day_revenue" : "TARGET"}, inplace=True)

C:\Users\omerf\AppData\Local\Temp\ipykernel_17256\2988388147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predictions_df_df.rename(columns={"15_day_revenue" : "TARGET"}, inplace=True)


In [201]:
id_range = pd.DataFrame({'ID': range(878594, 1463424)})

In [202]:
id_range

,ID
0,878594
1,878595
2,878596
3,878597
4,878598
...,...
584825,1463419
584826,1463420
584827,1463421
584828,1463422


In [203]:
nan_targets_df = id_range.copy()

In [204]:
nan_targets_df['TARGET'] = pd.NA

In [205]:
nan_targets_df

,ID,TARGET
0,878594,<NA>
1,878595,<NA>
2,878596,<NA>
3,878597,<NA>
4,878598,<NA>
...,...,...
584825,1463419,<NA>
584826,1463420,<NA>
584827,1463421,<NA>
584828,1463422,<NA>


In [206]:
nan_targets_df = nan_targets_df.set_index('ID').combine_first(predictions_df.set_index('ID')).reset_index()

In [207]:
nan_targets_df

,ID,TARGET
0,878594,0.037927
1,878595,NaN
2,878596,NaN
3,878597,0.53056
4,878598,0.039671
...,...,...
585172,1464309,0.25629
585173,1464314,3.172687
585174,1464315,1.255875
585175,1464317,0.442146


In [208]:
nan_targets_df = nan_targets_df.set_index('ID').combine_first(predictions_df_df.set_index('ID')).reset_index()

In [211]:
nan_targets_df["TARGET"].max()

250.69911809056967

In [210]:
nan_targets_df.to_csv("final.csv", index=False)